# D210 Data Preparation

### Data import

In [1]:
import pandas as pd
import numpy as np

# Read our hospital data CSV, keep only CaseOrder, State, Children, Age, Income, Marital, Gender, ReAdmis, VitD_level
med = pd.read_csv('medical_clean.csv', usecols=['CaseOrder', 'State', 'Children', 'Age', 'Income', 'Marital', 'Gender', 'ReAdmis', 'VitD_levels'])

# Read CMS Hospital Readmissions data, keep Facility ID, State, Readmission Measure, Readmission Rate
hosp = pd.read_csv('FY_2023_Hospital_Readmissions_Reduction_Program_Hospital.csv', usecols=[1,2,3,7])

# Read NHANES 2017-18 Vit D Study data, keep sequence number (name as 'CaseOrder'), Vit D blood serum level
vitd = pd.read_csv('VitD.csv', header=0, names=['CaseOrder', 'Vitamin D blood level (nmol/L)'], usecols=[0,1], na_values=['0'])

# Read NHANES 2017-18 Demographic data, keep sequence number (name as 'CaseOrder'), Gender, Age, Marital, Children (2 columns will need to be combined), Income
vitd_demo = pd.read_csv('VitD_demog.csv', usecols=[0,3,4,17,31,32,44])

print(med.info())
print(hosp.info())
print(vitd.info())
print(vitd_demo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CaseOrder    10000 non-null  int64  
 1   State        10000 non-null  object 
 2   Children     10000 non-null  int64  
 3   Age          10000 non-null  int64  
 4   Income       10000 non-null  float64
 5   Marital      10000 non-null  object 
 6   Gender       10000 non-null  object 
 7   ReAdmis      10000 non-null  object 
 8   VitD_levels  10000 non-null  float64
dtypes: float64(2), int64(3), object(4)
memory usage: 703.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18990 entries, 0 to 18989
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Facility ID                 18990 non-null  int64  
 1   State                       18990 non-null  object 
 2   Measure Name         

In [2]:
# Drop rows with null data & convert data types
med=med.dropna().convert_dtypes()
hosp=hosp.dropna().convert_dtypes()
vitd=vitd.dropna().convert_dtypes()
# Will convert dtypes manually in order to decode later
vitd_demo=vitd_demo.dropna()
vitd_demo=vitd_demo.astype({'SEQN': int, 'RIAGENDR': int, 'RIDAGEYR': int, 'DMDMARTL': int, 'DMDHHSZA':int, 'DMDHHSZB':int, 'INDFMIN2':int}) 

In [3]:
print(vitd_demo.info())
vitd_demo.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254 entries, 0 to 9253
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   SEQN      9254 non-null   int32
 1   RIAGENDR  9254 non-null   int32
 2   RIDAGEYR  9254 non-null   int32
 3   DMDMARTL  9254 non-null   int32
 4   DMDHHSZA  9254 non-null   int32
 5   DMDHHSZB  9254 non-null   int32
 6   INDFMIN2  9254 non-null   int32
dtypes: int32(7)
memory usage: 325.3 KB
None


,SEQN,RIAGENDR,RIDAGEYR,DMDMARTL,DMDHHSZA,DMDHHSZB,INDFMIN2
0,93703,2,2,0,3,0,15
1,93704,1,2,0,2,0,15
2,93705,2,66,3,0,0,3
3,93706,1,18,0,0,0,0
4,93707,1,13,0,0,3,10


#### Re-express coded variables categorically
Based on data dictionary at https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.htm

In [4]:
dict_gender = {1: 'Male', 2: 'Female'}
  
dict_marital = {0: np.nan, 1: 'Married', 2: 'Widowed', 3: 'Divorced', 4: 'Separated', 5: 'Never Married', 6: np.nan, 77: np.nan}

# NHANES uses codes for ranges of incomes - set to midpoint of range
dict_inc = {0: np.nan, 1: 2500, 2: 7500, 3: 12500, 4: 17500, 5: 22500, 6: 30000, 7: 40000, 8: 50000, 9: 60000, 10: 70000, 12: 35000, 13: 10000, 14: 87500, 15: 150000, 77: np.nan, 99: np.nan}

replace_dict = {'RIAGENDR': dict_gender, 'DMDMARTL': dict_marital, 'INDFMIN2': dict_inc}
vitd_demo.replace(replace_dict, inplace=True)


In [5]:
vitd_demo.head(20)

,SEQN,RIAGENDR,RIDAGEYR,DMDMARTL,DMDHHSZA,DMDHHSZB,INDFMIN2
0,93703,Female,2,NaN,3,0,150000.0
1,93704,Male,2,NaN,2,0,150000.0
2,93705,Female,66,Divorced,0,0,12500.0
3,93706,Male,18,NaN,0,0,NaN
4,93707,Male,13,NaN,0,3,70000.0
5,93708,Female,66,Married,0,0,30000.0
6,93709,Female,75,Widowed,0,0,7500.0
7,93710,Female,0,NaN,1,0,150000.0
8,93711,Male,56,Married,0,0,150000.0
9,93712,Male,18,NaN,0,2,17500.0


In [6]:
vitd_demo.rename(columns={'SEQN': 'CaseOrder', 'RIAGENDR': 'Gender', 'RIDAGEYR': 'Age', 'DMDMARTL': 'Marital', 'DMDHHSZA':'Children_under6', 'DMDHHSZB':'Children_6over', 'INDFMIN2':'Income'}, inplace=True)
vitd_demo.head()

,CaseOrder,Gender,Age,Marital,Children_under6,Children_6over,Income
0,93703,Female,2,NaN,3,0,150000.0
1,93704,Male,2,NaN,2,0,150000.0
2,93705,Female,66,Divorced,0,0,12500.0
3,93706,Male,18,NaN,0,0,NaN
4,93707,Male,13,NaN,0,3,70000.0


In [7]:
vitd_demo['Children'] = vitd_demo.Children_under6 + vitd_demo.Children_6over
vitd_demo.head(20)

,CaseOrder,Gender,Age,Marital,Children_under6,Children_6over,Income,Children
0,93703,Female,2,NaN,3,0,150000.0,3
1,93704,Male,2,NaN,2,0,150000.0,2
2,93705,Female,66,Divorced,0,0,12500.0,0
3,93706,Male,18,NaN,0,0,NaN,0
4,93707,Male,13,NaN,0,3,70000.0,3
5,93708,Female,66,Married,0,0,30000.0,0
6,93709,Female,75,Widowed,0,0,7500.0,0
7,93710,Female,0,NaN,1,0,150000.0,1
8,93711,Male,56,Married,0,0,150000.0,0
9,93712,Male,18,NaN,0,2,17500.0,2


In [8]:
vitd_demo.drop(axis=1, columns=['Children_under6', 'Children_6over'], inplace=True)

In [9]:
vitd_demo.head(20)

,CaseOrder,Gender,Age,Marital,Income,Children
0,93703,Female,2,NaN,150000.0,3
1,93704,Male,2,NaN,150000.0,2
2,93705,Female,66,Divorced,12500.0,0
3,93706,Male,18,NaN,NaN,0
4,93707,Male,13,NaN,70000.0,3
5,93708,Female,66,Married,30000.0,0
6,93709,Female,75,Widowed,7500.0,0
7,93710,Female,0,NaN,150000.0,1
8,93711,Male,56,Married,150000.0,0
9,93712,Male,18,NaN,17500.0,2


In [10]:
nhanes = vitd.merge(vitd_demo, how='left', on='CaseOrder', sort=True)
nhanes.head(20)

,CaseOrder,Vitamin D blood level (nmol/L),Gender,Age,Marital,Income,Children
0,93704,74.4,Male,2,NaN,150000.0,2
1,93705,89.9,Female,66,Divorced,12500.0,0
2,93706,53.8,Male,18,NaN,NaN,0
3,93707,58.2,Male,13,NaN,70000.0,3
4,93708,116.0,Female,66,Married,30000.0,0
5,93709,72.8,Female,75,Widowed,7500.0,0
6,93711,165.0,Male,56,Married,150000.0,0
7,93712,59.9,Male,18,NaN,17500.0,2
8,93713,63.5,Male,67,Divorced,30000.0,0
9,93714,47.5,Female,54,Married,40000.0,1


In [11]:
nhanes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7409 entries, 0 to 7408
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CaseOrder                       7409 non-null   Int64  
 1   Vitamin D blood level (nmol/L)  7409 non-null   Float64
 2   Gender                          7409 non-null   object 
 3   Age                             7409 non-null   int32  
 4   Marital                         4532 non-null   object 
 5   Income                          6795 non-null   float64
 6   Children                        7409 non-null   int32  
dtypes: Float64(1), Int64(1), float64(1), int32(2), object(2)
memory usage: 419.7+ KB


In [14]:
# Our medical data has VitD level in ng/mL, NHANES has nmol/L - convert to nmol/L
med['Vitamin D blood level (nmol/L)'] = med.VitD_levels * 2.5
med.drop(axis=1, columns='VitD_levels', inplace=True)
med.head()

,CaseOrder,State,Children,Age,Income,Marital,Gender,ReAdmis,Vitamin D blood level (nmol/L)
0,1,AL,1,53,86575.93,Divorced,Male,No,47.853664
1,2,FL,3,51,46805.99,Married,Female,No,47.350881
2,3,SD,3,53,14370.14,Widowed,Female,No,45.143768
3,4,MN,0,78,39741.49,Married,Male,No,41.442145
4,5,VA,1,22,1209.56,Widowed,Female,No,43.597673


In [15]:
allmed = med.merge(nhanes, how='outer', sort=True)
allmed

,CaseOrder,State,Children,Age,Income,Marital,Gender,ReAdmis,Vitamin D blood level (nmol/L)
0,1,AL,1,53,86575.93,Divorced,Male,No,47.853664
1,2,FL,3,51,46805.99,Married,Female,No,47.350881
2,3,SD,3,53,14370.14,Widowed,Female,No,45.143768
3,4,MN,0,78,39741.49,Married,Male,No,41.442145
4,5,VA,1,22,1209.56,Widowed,Female,No,43.597673
...,...,...,...,...,...,...,...,...,...
17404,102952,<NA>,0,70,17500.0,Married,Female,<NA>,82.7
17405,102953,<NA>,0,42,35000.0,Separated,Male,<NA>,49.5
17406,102954,<NA>,4,41,70000.0,Never Married,Female,<NA>,50.5
17407,102955,<NA>,1,14,60000.0,NaN,Female,<NA>,61.9


In [16]:
union = pd.concat([allmed, hosp])
union

,CaseOrder,State,Children,Age,Income,Marital,Gender,ReAdmis,Vitamin D blood level (nmol/L),Facility ID,Measure Name,Predicted Readmission Rate
0,1,AL,1,53,86575.93,Divorced,Male,No,47.853664,<NA>,<NA>,<NA>
1,2,FL,3,51,46805.99,Married,Female,No,47.350881,<NA>,<NA>,<NA>
2,3,SD,3,53,14370.14,Widowed,Female,No,45.143768,<NA>,<NA>,<NA>
3,4,MN,0,78,39741.49,Married,Male,No,41.442145,<NA>,<NA>,<NA>
4,5,VA,1,22,1209.56,Widowed,Female,No,43.597673,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
18866,<NA>,TX,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,670122,READM-30-HF-HRRP,21.7744
18867,<NA>,TX,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,670122,READM-30-COPD-HRRP,20.1507
18868,<NA>,TX,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,670122,READM-30-CABG-HRRP,13.2609
18869,<NA>,TX,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,670122,READM-30-AMI-HRRP,14.1772


In [17]:
_ = union.to_csv('unified_data.csv')

In [18]:
union.head(10)

,CaseOrder,State,Children,Age,Income,Marital,Gender,ReAdmis,Vitamin D blood level (nmol/L),Facility ID,Measure Name,Predicted Readmission Rate
0,1,AL,1,53,86575.93,Divorced,Male,No,47.853664,<NA>,<NA>,<NA>
1,2,FL,3,51,46805.99,Married,Female,No,47.350881,<NA>,<NA>,<NA>
2,3,SD,3,53,14370.14,Widowed,Female,No,45.143768,<NA>,<NA>,<NA>
3,4,MN,0,78,39741.49,Married,Male,No,41.442145,<NA>,<NA>,<NA>
4,5,VA,1,22,1209.56,Widowed,Female,No,43.597673,<NA>,<NA>,<NA>
5,6,OK,3,76,81999.88,Never Married,Male,No,49.031615,<NA>,<NA>,<NA>
6,7,OH,0,50,10456.05,Never Married,Male,No,36.879218,<NA>,<NA>,<NA>
7,8,VA,7,40,38319.29,Divorced,Female,No,49.221682,<NA>,<NA>,<NA>
8,9,FL,0,48,55586.48,Widowed,Male,No,49.133299,<NA>,<NA>,<NA>
9,10,MN,2,78,38965.22,Never Married,Female,No,45.560811,<NA>,<NA>,<NA>
